In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'tuning')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
# Alternate palette for promoter panel
metadata_alt_dict = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'promoters').set_index('construct').to_dict('dict')
promoter_palette = metadata_alt_dict['color']
promoter_markers = metadata_alt_dict['markers']

# Alternate palette for initial data
metadata_alt2_dict = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'designs').set_index('construct').to_dict('dict')
initial_palette = metadata_alt_dict['color']
initial_markers = metadata_alt_dict['markers']

miR/TS characterization data (`data2`) from Emma

In [ ]:
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'miR_characterization')

In [ ]:
# Calculate fold changes for orthogonal or matched conditions
by = ['condition','miR_construct','ts_construct']

# orthogonal exp: {miR} x {TS}x2
orthogonal_exp = ['ELP_exp09', 'ELP_exp12', 'ELP_exp13']
fcs_orthogonal = stats2[(stats2['exp'].isin(orthogonal_exp))].groupby(by=by)[['output_gmean_fc']].apply('mean').reset_index()
fcs_orthogonal = fcs_orthogonal.merge(metadata2, how='left', on=by)

# matched exp: {miR} x TSx{n}
matched_exp = ['ELP_exp11','ELP_exp49', 'ELP_exp50',]
fcs_matched = stats2[(stats2['exp'].isin(matched_exp))].groupby(by=by)[['output_gmean_fc']].apply('mean').reset_index()
fcs_matched = fcs_matched.merge(metadata2, how='left', on=by)

In [ ]:
# Create color palettes for miR/ts characterization
metadata2['color'] = base.colors['gray']
metadata2['matched'] = metadata2['miR'] == metadata2['ts']
metadata2.loc[metadata2['matched'], 'color'] = base.colors['green']
metadata2.loc[metadata2['ts']=='none', 'color'] = 'black'

metadata_comb_dict = metadata2.set_index('condition').to_dict('dict')
matched_palette = metadata_comb_dict['color']

metadata2['color'] = 'black'
metadata2.loc[metadata2['ts_num']>0, 'color'] = base.colors['green']
metadata2.loc[metadata2['ts_num']==2, 'color'] = metadata2.loc[metadata2['ts_num']==2, 'color'].apply(base.get_light_color)
metadata2.loc[metadata2['ts_num']==4, 'color'] = metadata2.loc[metadata2['ts_num']==4, 'color'].apply(base.get_light_color).apply(base.get_light_color)

metadata_comb_dict = metadata2.set_index('condition').to_dict('dict')
ts_num_palette = metadata_comb_dict['color']

Plasmid titration (`data3`)

In [ ]:
data3, quantiles3, stats3, metadata3 = base.load_data(base_path, metadata_path, 'plasmid_titration')

In [ ]:
greys = matplotlib.colors.ListedColormap(matplotlib.colormaps['Greys'](np.linspace(0.4,1,256)))
teals = sns.light_palette(base.colors['teal'], 4)[1:] + sns.dark_palette(base.colors['teal'], 4, reverse=True)[1:-1]
amounts = sorted([float(d) for d in data3['amount'].unique() if not np.isnan(d)])
teals_palette = dict(zip(amounts, teals))

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs
annotate_kwargs = base.annotate_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['full'],8))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 12, figure=fig, wspace=0.4, hspace=0.4, 
                                            height_ratios=[1.75,1.75,1.5,1.5,1.5], width_ratios=[1]*6+[0.9,1.1]+[1]*4)

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:2]),
    'B': fig.add_subfigure(fig_gridspec[0,3:9]),
    'C': fig.add_subfigure(fig_gridspec[0,9:]),
    'D': fig.add_subfigure(fig_gridspec[1,:6]),
    'E': fig.add_subfigure(fig_gridspec[1,6:]),
    'F': fig.add_subfigure(fig_gridspec[2,:2]),
    'G': fig.add_subfigure(fig_gridspec[2,2:6]),
    'H': fig.add_subfigure(fig_gridspec[2,7:11]),
    'I': fig.add_subfigure(fig_gridspec[3,:8]),
    'J': fig.add_subfigure(fig_gridspec[4,:6]),
    'K': fig.add_subfigure(fig_gridspec[4,6:9]),
    'L': fig.add_subfigure(fig_gridspec[4,9:]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_tuning-supp'/'fig_tuning-supp.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.35, top=0.35, right=0.15)
axes = subfig.subplots(1,1,)

plot_df = stats[(((stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)
display(plot_df['construct'].unique())

# stat CV
ax = axes
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=initial_palette,
                  ax=ax, marker=initial_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlim=xlim, xlabel='', ylabel='', ylim=(0,1.8))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, ax = plt.subplots(1,1, figsize=(3,2))
pairs = [('base','CL'), ('OL','CL')]
stat = 'output_variation'
sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=initial_palette, legend=False,)
annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.39, top=0.35, right=0.05)
ax = subfig.subplots(1,1)

fcs = fcs_orthogonal
plot_df = fcs[(fcs['ts']!='na') & (fcs['miR']!='na') & (fcs['miR_promoter']=='hPGK.d') 
              & (fcs['ts_num'].isin([0,2]))].pivot(index='miR', columns='ts', values='output_gmean_fc')
sns.heatmap(plot_df, annot=True, fmt='.2f', cmap=sns.light_palette(base.colors['green'], as_cmap=True), ax=ax, cbar=False)
ax.set(xlabel='target sites (x2)', ylabel='microRNA', title='Relative target expression')

# outline matched target sites
for i in range(len(plot_df)-1):
    ax.add_patch(matplotlib.patches.Rectangle((i, i), 1, 1, fill=False, edgecolor='black', lw=1))

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['K']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.43, top=0.35, right=0.15)
ax = subfig.subplots(1,1)

fcs = fcs_matched
fcs['ts_num'] = fcs['ts_num'].astype(int)
plot_df = fcs[(fcs['miR_promoter']=='hPGK.d') & (fcs['miR']!='none')].pivot(index='miR', columns='ts_num', values='output_gmean_fc')
sns.heatmap(plot_df, annot=True, fmt='.2f', cmap=sns.light_palette(base.colors['green'], as_cmap=True), ax=ax, cbar=False)
ax.set(xlabel='number of matched\ntarget sites', ylabel='microRNA', title='Rel. target expression')

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.52, top=0.35, right=0.15)
axes = subfig.subplots(1,5, sharey=True)

plot_df = stats2[(stats2['ts']!='na') & (stats2['miR']!='na') & (stats2['miR_promoter']=='hPGK.d') 
                 & (stats2['ts_num'].isin([0,2])) & (stats2['exp'].isin(orthogonal_exp))]

for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.pointplot(d, x='ts', y='output_gmean', hue='condition', palette=matched_palette, ax=ax, **scatter_kwargs)
    ax.set(yscale='log', xlabel='', title=f'{miR}', ylim=(4e2,4e4))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.minorticks_off()

axes[2].set(xlabel='target site (x2)')
ylabel_bbox = axes[0].get_yticklabels()[0]
title_bbox = axes[0].set_title(axes[0].get_title())
axes[0].annotate(text='miR:', xy=(1, 0.5), xycoords=(ylabel_bbox, title_bbox), 
                 ha="right", va="center", fontsize=base.font_sizes['base_size'],)
axes[0].set(ylabel='target mean')
        
fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,5, sharey=True, figsize=(5,2))
for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.stripplot(d, x='ts', y='output_gmean', hue='condition', palette=matched_palette, ax=ax, legend=False)
    ax.set(yscale='log', xlabel='', title=f'{miR}', ylim=(4e2,4e4))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    if miR != 'none':
        pairs = [('none',miR)]
        annotator = Annotator(ax, pairs, data=d, x='ts', y='output_gmean', **scatter_kwargs)
        annotator.configure(**annotate_kwargs).apply_and_annotate()

f, axes = plt.subplots(1,5, sharey=True, figsize=(5,2))
for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.stripplot(d, x='ts', y='output_gmean', hue='condition', palette=matched_palette, ax=ax, legend=False)
    ax.set(yscale='log', xlabel='', title=f'{miR}', ylim=(4e2,1e5))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    if miR != 'none':
        pairs = [('none',m) for m in d['ts'].unique() if (m != miR) and (m != 'none')]
        annotator = Annotator(ax, pairs, data=d, x='ts', y='output_gmean', **scatter_kwargs)
        annotator.configure(**annotate_kwargs).apply_and_annotate()

In [ ]:
subfig = subfigures['J']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.43, top=0.35, right=0.15)
axes = subfig.subplots(1,4, sharey=True)

plot_df = stats2[(stats2['miR_promoter']=='hPGK.d') & (stats2['miR']!='none') & (stats2['exp'].isin(matched_exp))]

for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.pointplot(d, x='ts_num', y='output_gmean_fc', hue='condition', palette=ts_num_palette, ax=ax, **scatter_kwargs)
    ax.set(yscale='log', xlabel='', title=f'{miR}', ylim=(3.5e-2,3e0))
    ax.axhline(1, c='black', ls=':')

axes[1].set_xlabel('number of matched target sites', x=1, ha='center')
ylabel_bbox = axes[0].get_yticklabels()[0]
title_bbox = axes[0].set_title(axes[0].get_title())
axes[0].annotate(text='miR:', xy=(1, 0.5), xycoords=(ylabel_bbox, title_bbox),
            ha="right", va="center", fontsize=base.font_sizes['base_size'], )
axes[0].set(ylabel='rel. target mean')

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,4, sharey=True, figsize=(5,2))
pairs = [(0,1), (0,2), (0,4)]
for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.stripplot(d, x='ts_num', y='output_gmean_fc', hue='condition', palette=matched_palette, ax=ax, legend=False)
    ax.set(yscale='log', xlabel='', title=f'{miR}', ylim=(3e-2,3e0))

    annotator = Annotator(ax, pairs, data=d, x='ts_num', y='output_gmean_fc', **scatter_kwargs)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.55, top=0.65, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# EF1a miR controls
miR_order = ['none', 'miR.FF5', 'miR.FF4', 'miRE.FF5', 'miRE.FF4',]
plot_df = stats[((((stats['group']=='miR')) & (stats['miR_loc']=='CDS')) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['construct'].unique())-0.5)
display(plot_df['construct'].unique())

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(3e1,8e4))
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(4e2,1e5))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0.5,1.3), ylabel='',)

for ax in axes:
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('none',miR) for miR in miR_order[1:]]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='miR', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    annotator = Annotator(ax, pairs, data=plot_df, x='miR', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()
axes[0].set(yscale='log')
axes[1].set(yscale='log')

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.55, top=0.65, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# EF1a miR controls
ts_order = ['none','FF3x1','FF4x1','FF5x1','FF6x1']
plot_df = stats[((((stats['group']=='ts3')) & (stats['ts_num']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
xlim = (-0.5, len(plot_df['ts'].unique())-0.5)
display(plot_df['construct'].unique())

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(3e1,8e4))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,4e4))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0.5,1.4), ylabel='',)

for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    sns.despine(ax=ax)
    ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('none',ts) for ts in ts_order[1:]]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    annotator = Annotator(ax, pairs, data=plot_df, x='ts', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()
axes[0].set(yscale='log')
axes[1].set(yscale='log')

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.35, top=0.35, right=0.15)
ax = subfig.subplots(1,1,)

miR_order = ['none','miR.FF4','miR.FF5','miRE.FF4','miRE.FF5']
plot_df = stats[(((stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['miR'].unique())-0.5)
display(plot_df['construct'].unique())

# stat std
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(xlim=xlim, xlabel='', ylabel='', title='Standard deviation', yscale='log', ylim=(4e2, 5e4))
ax.minorticks_off()
ax.set_xticklabels([l.get_text().replace('.','-\n') for l in ax.get_xticklabels()])

fig.savefig(rd.outfile(output_path))

# perform statistical tests
pairs = [('base','CL'), ('OL','CL')]
f, axes = plt.subplots(1,len(miR_order)-1, figsize=(5,2), gridspec_kw=dict(wspace=0.5))
for ax, miR in zip(axes, miR_order[1:]):
    print('microRNA: '+miR)
    test_df = plot_df[(plot_df['miR']==miR) | (plot_df['group']=='base')]
    sns.stripplot(data=test_df, x='ts_label', y='output_std', ax=ax, hue='construct', palette=main_palette, legend=False)
    ax.set(ylabel='', xlabel='', title=miR.replace('.','-'), yscale='log')

    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y='output_std')
    annotator.configure(**annotate_kwargs,).apply_and_annotate()
    print('\n')
axes[0].set(ylabel='Standard deviation')

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.35, top=0.35, right=0.15)
ax = subfig.subplots(1,1,)

promoter_order = ['EF1a','CAG','EFS','hPGK']
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) | (stats['group']=='base'))].copy()
plot_df['promoter'] = plot_df['promoter'].astype(pd.api.types.CategoricalDtype(categories=promoter_order, ordered=True))
plot_df.sort_values(['promoter','group','ts_kind'], inplace=True)
display(plot_df['construct'].unique())

# remove outlier hPGK point
plot_df = plot_df[~((plot_df['promoter']=='hPGK') & (plot_df['exp']=='exp92') & (plot_df['biorep']==1))]

# shift xticks to add more space between promoter groups
buffer = 0.6
xlim = (-0.5, len(plot_df['construct'].unique())-0.5+buffer*(len(promoter_order)-1))
xtick_locs = np.concatenate([[x+i*(3+buffer) for x in range(3)] for i in range(len(promoter_order))])
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)

# stat std
for construct, group in plot_df.groupby('construct', sort=False):
    sns.pointplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=promoter_palette,
                  ax=ax, marker=promoter_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(xlim=xlim, xlabel='', ylabel='', title='Standard deviation', yscale='log', xticks=xtick_locs, 
       ylim=(7e1,3e5), yticks=(1e2,1e3,1e4,1e5))

# add shaded regions every other promoter
span1 = (xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2, xtick_locs[5]+(xtick_locs[6]-xtick_locs[5])/2,)
span2 = (xtick_locs[8]+(xtick_locs[9]-xtick_locs[8])/2, xtick_locs[11]+(1+buffer)/2,)
ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
ax.axvspan(*span2, color=base.get_light_color(base.colors['gray']), alpha=0.2,)

ax.set(xticklabels=['']*len(ax.get_xticklabels()))
ax.minorticks_off()

# add promoter labels
yloc = -6
for j, prom in enumerate(promoter_order):
    ax.annotate(prom.replace('a','α'), (1+j*(3+buffer),yloc), xycoords=('data','axes points'), ha='center', va='top', fontsize=base.font_sizes['smaller_size'])

fig.savefig(rd.outfile(output_path))

# perform statistical tests
pairs = [('base','CL'), ('OL','CL')]
f, axes = plt.subplots(1,len(promoter_order), figsize=(5,2), gridspec_kw=dict(wspace=0.5))
for ax, promoter in zip(axes, promoter_order):
    print('Promoter: '+promoter)
    test_df = plot_df[(plot_df['promoter']==promoter)]
    sns.stripplot(data=test_df, x='ts_label', y='output_std', ax=ax, hue='construct', palette=promoter_palette, legend=False)
    ax.set(ylabel='', xlabel='', title=promoter, yscale='log')

    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y='output_std')
    annotator.configure(**annotate_kwargs,).apply_and_annotate()
    print('\n')
axes[0].set(ylabel='Standard deviation')

In [ ]:
subfig = subfigures['L']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.43, top=0.35, right=0.1)
ax = subfig.subplots(1,1,)

plot_df = stats[(stats['group']=='controller') & (stats['miR']=='miRE.FF4') & (stats['design']==1) & (stats['promoter']=='EF1a')].copy()
xlim = (-0.5, len(plot_df['ts_num'].unique())-0.5)
display(plot_df['construct'].unique())

# stat std
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_num', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(xlim=xlim, xlabel='# target sites', ylabel='', title='Standard deviation', yscale='log', ylim=(3e2,4e4))
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

# perform statistical tests
pairs = [('OL','CL')]
f, axes = plt.subplots(1,len(plot_df['ts_num'].unique()), figsize=(5,2), gridspec_kw=dict(wspace=0.5))
for ax, ts_num in zip(axes, plot_df['ts_num'].unique()):
    print('Number of target sites: '+str(ts_num))
    test_df = plot_df[(plot_df['ts_num']==ts_num)]
    sns.stripplot(data=test_df, x='ts_label', y='output_std', ax=ax, hue='construct', palette=promoter_palette, legend=False)
    ax.set(ylabel='', xlabel='', title=str(ts_num), yscale='log')

    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y='output_std')
    annotator.configure(**annotate_kwargs,).apply_and_annotate()
    print('\n')
axes[0].set(ylabel='Standard deviation')

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.4, top=0.35, right=0.25)
axes = subfig.subplots(1,3, gridspec_kw=dict(wspace=0.4))

plot_df = stats3[stats3['construct']!='AIO.039']
line_kwargs2 = dict(legend=False, dashes=False, markersize=4, markeredgecolor='white', markeredgewidth=0.5)

# stats gmean vs amount
ax = axes[0]
sns.lineplot(data=plot_df, x='amount', y='output_gmean', hue='construct', palette=main_palette, ax=ax,
             style='construct', markers=main_markers, **line_kwargs2)
ax.set(yscale='log', ylabel='', title='Mean',
       xlabel='rel. plasmid amount', xticks=[0.5**i for i in range(5)]+[0], xticklabels=['1.0','0.5','','','',''], xlim=(0,1.1))
ax.minorticks_off()

# stats std vs amount
ax = axes[1]
sns.lineplot(data=plot_df, x='amount', y='output_std', hue='construct', palette=main_palette, ax=ax,
             style='construct', markers=main_markers, **line_kwargs2)
ax.set(yscale='log', ylabel='', title='Std.',
       xlabel='rel. plasmid amount', xticks=[0.5**i for i in range(5)]+[0], xticklabels=['1.0','0.5','','','',''], xlim=(0,1.1))
ax.minorticks_off()

# stats CV vs amount
ax = axes[2]
sns.lineplot(data=plot_df, x='amount', y='output_variation', hue='construct', palette=main_palette, ax=ax,
             style='construct', markers=main_markers, **line_kwargs2)
ax.set(ylim=(0,2.2), ylabel='', title='CV',
       xlabel='rel. plasmid amount', xticks=[0.5**i for i in range(5)]+[0], xticklabels=['1.0','0.5','','','',''], xlim=(0,1.1))

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_tuning-supp.pdf'))